**Import Essential Libraries**

In [ ]:
import ee
import geemap as gee
import xarray as xr
import matplotlib.pyplot as plt

In [ ]:
import xee

**Project Authentification and Initialization**

Insert ProjectID from Earth Engine

In [ ]:
ee.Authenticate()
ee.Initialize(project = 'ee-vindevon11', opt_url='https://earthengine-highvolume.googleapis.com')

In [ ]:
map = gee.Map(basemap = 'TERRAIN')
map

In [ ]:
roi = map.draw_last_feature.geometry()
map.center_object(roi)

**VNP09GA: VIIRS Surface Reflectance Daily 500m and 1km**

Computing NDVI using this VIIRS Dataset will require utilizing Bands I1 and I2

In [ ]:
time_start = '2025-01-01'
time_end = '2025-02-01'

In [ ]:
sr = ee.ImageCollection("NASA/VIIRS/002/VNP09GA").filterDate(time_start, time_end).select('I1', 'I2')

In [ ]:
def ndvi(img):
  index = img.normalizedDifference(['I2','I1']).rename('ndvi')
  return index.copyProperties(img, img.propertyNames())

In [ ]:
viirs_ndvi = sr.map(ndvi)

In [ ]:
viirs_ndvi.getInfo()

Converting VIIRS NDVI Image Collection to an XArray Data Structure

In [ ]:
ds_ndvi  = xr.open_dataset(viirs_ndvi, engine = 'ee', crs = 'EPSG:4326', scale = 0.001, geometry = roi)

In [ ]:
ds_ndvi

**Visual Plot of LA Fires in January 2025**

In [ ]:
ds_ndvi.ndvi.plot(x = 'lon', y = 'lat', col = 'time', col_wrap = 7, robust = True, cmap = 'jet')
plt.savefig('ndvi_fire.png', dpi = 360, bbox_inches = 'tight')

**VNP14A1.002: Thermal Anomalies/Fire Daily L3 Global 1km SIN Grid**

In [ ]:
temp = ee.ImageCollection("NASA/VIIRS/002/VNP14A1").select('MaxFRP').filterDate(time_start, time_end)

In [ ]:
ds_temp = xr.open_dataset(temp, engine = 'ee', crs = 'EPSG:4326', scale = 0.005, geometry = roi)

In [ ]:
ds_temp

Visualizing Temperature Variations in LA in January 2025

In [ ]:
ds_temp.MaxFRP.plot(x = 'lon', y = 'lat', col = 'time', col_wrap = 6, robust = True, cmap = 'hot_r')

**VNP21A1D.002: Day Land Surface Temperature and Emissivity Daily 1km**

In [ ]:
lst = ee.ImageCollection("NASA/VIIRS/002/VNP21A1D").select('LST_1KM').filterDate(time_start, time_end)

In [ ]:
ds_lst = xr.open_dataset(lst, engine = 'ee', crs = 'EPSG:4326', scale = 0.005, geometry = roi)

In [ ]:
ds_lst

In [ ]:
ds_lst.LST_1KM.plot(x = 'lon', y = 'lat', col = 'time', col_wrap = 6, robust = True, cmap = 'Reds')